# GenAI Prompt Engineering Examples

---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [1]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform'),
    ('google.cloud.storage', 'google-cloud-storage'),
    ('google.cloud.bigquery', 'google-cloud-bigquery'),
    ('streamlit', 'streamlit')
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [2]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

inputs:

In [2]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'mg-ce-demos'

In [3]:
REGION = 'us-central1'

# Set the BUCKET name for saving work:
BUCKET = PROJECT_ID

packages:

In [4]:
import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from google.cloud import aiplatform
from google.cloud import bigquery
from google.cloud import storage

import vertexai.vision_models # Imagen Models
import vertexai.preview.vision_models
import vertexai.language_models # PaLM and Codey Models
import vertexai.generative_models # for Gemini Models

import json
import io
import base64
import asyncio
import requests
import IPython
import datetime, time

In [5]:
aiplatform.__version__

'1.71.1'

clients:

In [6]:
vertexai.init(project = PROJECT_ID, location = REGION)
gcs = storage.Client(project = PROJECT_ID)
bq = bigquery.Client(project = PROJECT_ID)

bucket = gcs.lookup_bucket(BUCKET)

In [7]:
# Gemini Models
gemini15_multimodal = vertexai.generative_models.GenerativeModel("gemini-1.5-pro-002")
gemini_flash = vertexai.generative_models.GenerativeModel("gemini-1.5-flash-002")

## Streamlit app

In [50]:
%%writefile geminibot.py

import streamlit as st
from google.cloud import aiplatform
import vertexai.generative_models
from vertexai.generative_models import GenerativeModel, Part, Content
import tempfile
import os
import mimetypes
from typing import Iterator

PROJECT_ID = 'mg-ce-demos'
REGION = 'us-central1'

# Initialize Vertex AI
vertexai.init(project = PROJECT_ID, location = REGION)

# Initialize Gemini 1.5 Pro model
model = vertexai.generative_models.GenerativeModel("gemini-1.5-pro-002")

def process_uploaded_file(uploaded_file) -> Part:
    """Process uploaded file and convert to Gemini Part object."""
    if uploaded_file is None:
        return None
        
    # Create a temporary file to store the uploaded content
    with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
        tmp_file.write(uploaded_file.getvalue())
        tmp_file_path = tmp_file.name

    # Determine mime type
    mime_type, _ = mimetypes.guess_type(uploaded_file.name)
    if mime_type is None:
        mime_type = "application/octet-stream"

    # Create Part object from file
    with open(tmp_file_path, "rb") as f:
        file_content = f.read()
        
    # Clean up temporary file
    os.unlink(tmp_file_path)
    
    return Part.from_data(data=file_content, mime_type=mime_type)

def stream_chat(chat_history: list, query: str, uploaded_part: Part = None) -> Iterator[str]:
    """Stream chat responses from Gemini."""
    # Initialize chat
    formatted_history = []
    for msg in chat_history:
        formatted_history.extend([
            Content(role="user", parts=[Part.from_text(str(msg["user"]))]),
            Content(role="model", parts=[Part.from_text(str(msg["model"]))])
        ])
    
    chat = model.start_chat(history=formatted_history)
    
    # Prepare message parts
    message_parts = [query]
    if uploaded_part:
        message_parts.insert(0, uploaded_part)
    
    # Get streaming response
    response = chat.send_message(message_parts, stream=True)
    
    for chunk in response:
        if hasattr(chunk, "text"):
            yield chunk.text

def main():
    st.title("Gemini 1.5 Pro Chat")

    # Add clear history button in the sidebar
    if st.sidebar.button("Clear Chat History"):
        st.session_state.chat_history = []
        st.rerun()
    
    # Initialize session state for chat history
    if "chat_history" not in st.session_state:
        st.session_state.chat_history = []
    
    # File uploader
    uploaded_file = st.file_uploader("Upload a file (optional)", type=["txt", "pdf", "png", "jpg", "jpeg"])
    
    # Convert uploaded file to Part if present
    uploaded_part = process_uploaded_file(uploaded_file) if uploaded_file else None
    
    # Chat input
    user_input = st.chat_input("Type your message here...")
    
    # Display chat history
    for message in st.session_state.chat_history:
        with st.chat_message("user"):
            st.write(message["user"])
        with st.chat_message("model"):
            st.write(message["model"])
    
    # Handle new user input
    if user_input:
        # Show user message
        with st.chat_message("user"):
            st.write(user_input)
            
        # Show assistant response with streaming
        with st.chat_message("model"):
            message_placeholder = st.empty()
            full_response = ""
            
            # Stream the response
            for chunk in stream_chat(st.session_state.chat_history, user_input, uploaded_part):
                full_response += chunk
                message_placeholder.markdown(full_response + "▌")
            
            # Update final response
            message_placeholder.markdown(full_response)
        
        # Update chat history
        st.session_state.chat_history.append({
            "user": user_input,
            "model": full_response
        })
        
if __name__ == "__main__":
    main()

Overwriting geminibot.py
